In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA
from sklearn.decomposition import FastICA
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.ensemble import IsolationForest
from sklearn.cluster import DBSCAN
import umap
import hdbscan

d:\Apps\anaconda3\envs\poop\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


### Wczytanie danych

In [2]:
hc_training = pd.read_csv('../../../classifier_data/HC_training.csv', sep='\t').T
hc_test = pd.read_csv('../../../classifier_data/HC_test.csv', sep='\t').T
oc_training = pd.read_csv('../../../classifier_data/OC_training.csv', sep='\t').T
oc_test = pd.read_csv('../../../classifier_data/OC_test.csv', sep='\t').T

synthetic_hc_data_path = '../../../best_models/HC/WGANGP/1000/synthetic_data/'
model_id = os.listdir(synthetic_hc_data_path)[0]
hc_synthetic = pd.read_csv(f'{synthetic_hc_data_path}/{model_id}/generated_data.tsv', sep='\t')

synthetic_oc_data_path = '../../../best_models/OC/WGANGP/1000/synthetic_data/'
model_id = os.listdir(synthetic_oc_data_path)[0]
oc_synthetic = pd.read_csv(f'{synthetic_oc_data_path}/{model_id}/generated_data.tsv', sep='\t')

# combine training and test data with labels
hc_training['label'] = 'HC'
hc_test['label'] = 'HC'
oc_training['label'] = 'OC'
oc_test['label'] = 'OC'

hc_synthetic['label'] = 'HC'
oc_synthetic['label'] = 'OC'

training_data = pd.concat([hc_training, oc_training])
test_data = pd.concat([hc_test, oc_test])

In [3]:
# keep only the columns that are in both datasets
# Znalezienie wspólnych kolumn
common_columns = training_data.columns.intersection(test_data.columns)
common_columns = common_columns.intersection(hc_synthetic.columns)
common_columns = common_columns.intersection(oc_synthetic.columns)

# Uporządkowanie kolumn w real_data
training_data = training_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
test_data = test_data[common_columns]

# Uporządkowanie kolumn w synthetic_data
hc_synthetic = hc_synthetic[common_columns]
oc_synthetic = oc_synthetic[common_columns]

common_columns

Index(['ENSG00000000938', 'ENSG00000002549', 'ENSG00000004487',
       'ENSG00000004534', 'ENSG00000005059', 'ENSG00000005700',
       'ENSG00000005844', 'ENSG00000005955', 'ENSG00000006744',
       'ENSG00000007392',
       ...
       'ENSG00000243955', 'ENSG00000244038', 'ENSG00000244734',
       'ENSG00000249072', 'ENSG00000255823', 'ENSG00000257207',
       'ENSG00000266714', 'ENSG00000269028', 'ENSG00000271043', 'label'],
      dtype='object', length=716)

In [4]:
# add synthetic data to training data
training_data_synthetic = pd.concat([training_data, hc_synthetic, oc_synthetic])
training_data_synthetic

,ENSG00000000938,ENSG00000002549,ENSG00000004487,ENSG00000004534,ENSG00000005059,ENSG00000005700,ENSG00000005844,ENSG00000005955,ENSG00000006744,ENSG00000007392,...,ENSG00000243955,ENSG00000244038,ENSG00000244734,ENSG00000249072,ENSG00000255823,ENSG00000257207,ENSG00000266714,ENSG00000269028,ENSG00000271043,label
VUMC-HC-0033-TR2591,4.432779,4.954165,4.211676,4.722403,4.399111,4.328301,4.211676,4.465409,4.125114,3.793786,...,5.112603,4.954165,6.995484,6.002046,6.095028,9.847484,4.465409,7.757440,6.525607,HC
Vumc-HD-70-TR1062,5.786592,6.072522,5.309622,5.479209,5.828302,5.215895,5.247890,5.743579,5.005196,4.546459,...,5.079361,5.908122,10.021953,5.215895,5.215895,8.262227,4.139052,6.645036,5.397116,HC
VUMC-HC0053-DOT-HD-48h-TR3087,5.526849,4.724280,4.292393,4.253528,4.913257,4.170728,4.934939,4.868709,4.956248,4.126378,...,4.588670,4.822471,7.050183,3.861049,3.861049,8.831693,4.030369,4.253528,3.720811,HC
Vumc-HD-149-TR932,4.923042,4.945413,4.727528,3.808559,4.673182,4.779470,3.937986,5.275333,4.923042,3.808559,...,4.146284,4.804614,8.661739,6.247257,6.360425,3.647372,3.808559,8.517492,6.826289,HC
Vumc-HD-36-TR1165,6.484772,4.751692,5.811822,7.053716,5.998264,4.402386,5.938859,6.443015,5.020424,5.998264,...,3.862107,5.596133,9.877834,5.515912,5.241598,3.862107,6.355612,7.025844,5.339481,HC
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76,4.066254,4.223423,3.891723,4.320584,3.841150,3.763954,3.715728,4.549974,3.551198,3.527600,...,3.884228,4.182802,10.362675,4.414996,4.673464,5.737569,3.633037,4.756148,3.875808,OC
77,4.928442,5.487600,5.080374,4.622335,4.659443,3.869087,4.052709,5.460585,4.046926,3.663866,...,4.147292,4.602828,10.606203,5.344729,5.860985,7.646285,4.108643,5.547580,4.605361,OC
78,4.054417,4.646725,4.587339,4.256235,3.864107,3.743585,3.711968,4.980480,3.899344,3.704812,...,4.134216,4.018330,10.244266,4.675424,5.712760,6.669411,3.704689,4.832443,4.099801,OC
79,5.027305,5.573289,5.466425,4.459724,5.166085,4.026987,4.261550,5.591042,4.705081,4.484023,...,4.836409,4.560970,9.298695,5.731542,5.748852,7.457558,4.448072,5.967367,4.572684,OC


### Wykorzystanie klasyfikatorów

In [5]:
# klasyfikacja z uyciem svm
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_auc_score


def svm_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)

    # Klasyfikacja przy użyciu SVM
    svm = SVC(kernel='linear', random_state=42)
    svm.fit(X_train, y_train)
    y_pred = svm.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))


svm_classification(training_data, test_data)
svm_classification(training_data_synthetic, test_data)

Dokładność: 0.9351351351351351
              precision    recall  f1-score   support

          HC       0.95      0.95      0.95       131
          OC       0.89      0.89      0.89        54

    accuracy                           0.94       185
   macro avg       0.92      0.92      0.92       185
weighted avg       0.94      0.94      0.94       185

[[125   6]
 [  6  48]]
Dokładność: 0.9351351351351351
              precision    recall  f1-score   support

          HC       0.95      0.95      0.95       131
          OC       0.89      0.89      0.89        54

    accuracy                           0.94       185
   macro avg       0.92      0.92      0.92       185
weighted avg       0.94      0.94      0.94       185

[[125   6]
 [  6  48]]


In [6]:
from sklearn.tree import DecisionTreeClassifier
# klasyfikacja z uyciem random forest
def decision_tree_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    dt = DecisionTreeClassifier(max_depth=5, random_state=42)
    dt.fit(X_train, y_train)
    y_pred = dt.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

decision_tree_classification(training_data, test_data)
decision_tree_classification(training_data_synthetic, test_data)

Dokładność: 0.8
              precision    recall  f1-score   support

          HC       0.83      0.90      0.86       131
          OC       0.70      0.56      0.62        54

    accuracy                           0.80       185
   macro avg       0.76      0.73      0.74       185
weighted avg       0.79      0.80      0.79       185

[[118  13]
 [ 24  30]]
Dokładność: 0.8
              precision    recall  f1-score   support

          HC       0.87      0.84      0.86       131
          OC       0.64      0.70      0.67        54

    accuracy                           0.80       185
   macro avg       0.76      0.77      0.76       185
weighted avg       0.81      0.80      0.80       185

[[110  21]
 [ 16  38]]


In [7]:
from sklearn.ensemble import RandomForestClassifier
# klasyfikacja z uyciem random forest
def random_forest_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    rf = RandomForestClassifier(n_estimators=100, random_state=42)
    rf.fit(X_train, y_train)
    y_pred = rf.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

random_forest_classification(training_data, test_data)
random_forest_classification(training_data_synthetic, test_data)

Dokładność: 0.8486486486486486
              precision    recall  f1-score   support

          HC       0.82      1.00      0.90       131
          OC       1.00      0.48      0.65        54

    accuracy                           0.85       185
   macro avg       0.91      0.74      0.78       185
weighted avg       0.88      0.85      0.83       185

[[131   0]
 [ 28  26]]
Dokładność: 0.8594594594594595
              precision    recall  f1-score   support

          HC       0.85      0.97      0.91       131
          OC       0.89      0.59      0.71        54

    accuracy                           0.86       185
   macro avg       0.87      0.78      0.81       185
weighted avg       0.86      0.86      0.85       185

[[127   4]
 [ 22  32]]


In [8]:
from xgboost import XGBClassifier

# klasyfikacja z uyciem xgboost
def xgboost_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    y_train = y_train.replace('HC', 0)
    y_train = y_train.replace('OC', 1)
    y_test = y_test.replace('HC', 0)
    y_test = y_test.replace('OC', 1)
    xgb = XGBClassifier(random_state=42)
    xgb.fit(X_train, y_train)
    y_pred = xgb.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

xgboost_classification(training_data, test_data)
xgboost_classification(training_data_synthetic, test_data)

Dokładność: 0.8972972972972973
              precision    recall  f1-score   support

           0       0.89      0.97      0.93       131
           1       0.91      0.72      0.80        54

    accuracy                           0.90       185
   macro avg       0.90      0.85      0.87       185
weighted avg       0.90      0.90      0.89       185

[[127   4]
 [ 15  39]]
Dokładność: 0.9027027027027027
              precision    recall  f1-score   support

           0       0.91      0.95      0.93       131
           1       0.88      0.78      0.82        54

    accuracy                           0.90       185
   macro avg       0.89      0.87      0.88       185
weighted avg       0.90      0.90      0.90       185

[[125   6]
 [ 12  42]]


In [9]:
from sklearn.neural_network import MLPClassifier

# klasyfikacja z uzyciem sieci neuronowych

def nn_classification(training_data, test_data):
    # Oddziel etykiety od cech
    X_train = training_data.drop('label', axis=1)
    y_train = training_data['label']
    X_test = test_data.drop('label', axis=1)
    y_test = test_data['label']

    # Normalizacja danych
    scaler = StandardScaler()
    X_train = scaler.fit_transform(X_train)
    X_test = scaler.transform(X_test)
    
    # Klasyfikacja
    nn = MLPClassifier(hidden_layer_sizes=(100,100), random_state=42)
    nn.fit(X_train, y_train)
    y_pred = nn.predict(X_test)

    # Wypisanie dokładności i innych metryk
    print(f"Dokładność: {accuracy_score(y_test, y_pred)}")
    print(classification_report(y_test, y_pred))
    print(confusion_matrix(y_test, y_pred))

nn_classification(training_data, test_data)
nn_classification(training_data_synthetic, test_data)

Dokładność: 0.9027027027027027
              precision    recall  f1-score   support

          HC       0.93      0.93      0.93       131
          OC       0.83      0.83      0.83        54

    accuracy                           0.90       185
   macro avg       0.88      0.88      0.88       185
weighted avg       0.90      0.90      0.90       185

[[122   9]
 [  9  45]]
Dokładność: 0.8972972972972973
              precision    recall  f1-score   support

          HC       0.91      0.95      0.93       131
          OC       0.86      0.78      0.82        54

    accuracy                           0.90       185
   macro avg       0.88      0.86      0.87       185
weighted avg       0.90      0.90      0.90       185

[[124   7]
 [ 12  42]]
